# import packages

In [1]:
import pandas as pd
import numpy as np
import json
import torch
from torch import nn
import math

# preprocess data

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# import os
# os.chdir('/content/drive/MyDrive/NLP2')

In [4]:
with open('data/english-train.json', 'r') as json_file:
    english_train = json.load(json_file)
with open('data/english-dev.json', 'r') as json_file:
    english_dev = json.load(json_file)
with open('data/english-test.json', 'r') as json_file:
    english_test = json.load(json_file)


In [5]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print("Device: ", device)

Device:  cuda:0


In [6]:
context_seq = []
patient_ques = []
docter_ans = []

context_seq.extend([i['description'] for i in english_train])
patient_ques.extend([i['utterances'][0] for i in english_train])
docter_ans.extend([i['utterances'][1] for i in english_train])

context_seq.extend([i['description'] for i in english_dev])
patient_ques.extend([i['utterances'][0] for i in english_dev])
docter_ans.extend([i['utterances'][1] for i in english_dev])

# Changed
# context_seq.extend([i['description'] for i in english_test])
# patient_ques.extend([i['utterances'][0] for i in english_test])
# docter_ans.extend([i['utterances'][1] for i in english_test])

context_seq = ['SOS '+i+' EOS' for i in context_seq]
patient_ques = ['SOS '+i[9:]+' EOS' for i in patient_ques]
docter_ans = ['SOS '+i[8:]+' EOS' for i in docter_ans]

tot_data = []
all_patient_ques = []
for i in range(len(patient_ques)):
  all_patient_ques.append(patient_ques[i]+" SEP "+context_seq[i])

tot_data.extend(all_patient_ques)
tot_data.extend(docter_ans)

In [7]:
# Changed
context_seq_test = []
patient_ques_test = []
docter_ans_test = []

context_seq_test.extend([i['description'] for i in english_test])
patient_ques_test.extend([i['utterances'][0] for i in english_test])
docter_ans_test.extend([i['utterances'][1] for i in english_test])

context_seq_test = ['SOS '+i+' EOS' for i in context_seq_test]
patient_ques_test = ['SOS '+i[9:]+' EOS' for i in patient_ques_test]
docter_ans_test = ['SOS '+i[8:]+' EOS' for i in docter_ans_test]

tot_data_test = []
all_patient_ques_test = []

for i in range(len(patient_ques_test)):
  all_patient_ques_test.append(patient_ques_test[i]+" SEP "+context_seq_test[i])

tot_data_test.extend(all_patient_ques_test)
tot_data_test.extend(docter_ans_test)

In [8]:
from transformers import AutoTokenizer, BertModel
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

patient_input = [tokenizer(patient_ques[idx],context_seq[idx], truncation=True, padding='max_length', max_length=512, return_tensors="pt") for idx in range(len(patient_ques))]
docter_input = [tokenizer(docter_ans[idx], truncation=True, padding='max_length', max_length=512, return_tensors="pt")["input_ids"] for idx in range(len(patient_ques))]

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
from torch.utils.data import Dataset, DataLoader
class CustomDataset(Dataset):
    def __init__(self, p_info,c_info,d_info):
        self.patient_ques = p_info
        self.context_seq = c_info
        self.doc_ans = d_info

    def __len__(self):
        return len(self.patient_ques)

    def __getitem__(self, idx):

        input_seq = tokenizer(self.patient_ques[idx],self.context_seq[idx], truncation=True, padding='max_length', max_length=512, return_tensors="pt")
        target_seq = tokenizer(self.doc_ans[idx], truncation=True, padding='max_length', max_length=512, return_tensors="pt")["input_ids"]

        return input_seq, target_seq

# Create dataset
dataset = CustomDataset(patient_ques,context_seq,docter_ans)

# Create DataLoader
batch_size = 8  # You can adjust this
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [10]:
for inputs, targets in dataloader:
  break
input_seq = tokenizer(patient_ques[1:5],context_seq[1:5], truncation=True, padding='max_length', max_length=512, return_tensors="pt")

In [11]:
input_seq['input_ids'].shape

torch.Size([4, 512])

In [12]:
inputs['input_ids'] = inputs['input_ids'].squeeze(1)
inputs['token_type_ids'] = inputs['token_type_ids'].squeeze(1)
inputs['attention_mask'] = inputs['attention_mask'].squeeze(1)

In [13]:
test_dataset = CustomDataset(patient_ques_test,context_seq_test,docter_ans_test)

# Create DataLoader
batch_size = 8  # You can adjust this
testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [14]:
# Initialize the optimizer
from transformers import BertModel, BertConfig

config = BertConfig.from_pretrained('bert-base-uncased')
model = BertModel(config)
model = model.to(device)

In [15]:
len(tokenizer.vocab)

30522

In [16]:
model.embeddings.word_embeddings.weight.shape

torch.Size([30522, 768])

In [17]:
# Replace Word Embedding by Graph Embedding -----------------------------------------------
wordembeddings = torch.from_numpy(np.load("../emb/node_embedding_768.npy"))

word2embidx = {}
with open("../raw/node_info_new.txt", "r") as f:
    for line in f.readlines():
        word, embidx = line.rstrip().split("\t")
        embidx = int(embidx)
        word2embidx[word] = embidx
        
# model = BERT(num_embeddings=len(tokenizer.word_index)).to(device)

replace_count = 0
for word in word2embidx:
    if word in tokenizer.vocab.keys():
        embidx = word2embidx[word]
        tokenidx = tokenizer.vocab[word]
        model.embeddings.word_embeddings.weight.data[tokenidx] = wordembeddings[embidx].to(device)
        replace_count += 1
        
print(replace_count, len(word2embidx))
# Done  -----------------------------------------------------------------------------------

315 862


In [18]:
from transformers import AdamW
from torch.nn import CrossEntropyLoss

In [19]:
linear_layer = torch.nn.Linear(config.hidden_size, config.vocab_size)
linear_layer.weight = model.embeddings.word_embeddings.weight  # weight tying
linear_layer = linear_layer.to(device)

# Training loop
optimizer = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
num_epochs = 100
for epoch in range(num_epochs):
    running_loss = 0
    for inputs, targets in dataloader:
        inputs,targets = inputs.to(device),targets.to(device)
        inputs['input_ids'] = inputs['input_ids'].squeeze(1)
        inputs['token_type_ids'] = inputs['token_type_ids'].squeeze(1)
        inputs['attention_mask'] = inputs['attention_mask'].squeeze(1)

        outputs = model(**inputs)
        logits = linear_layer(outputs.last_hidden_state)

        # Compute the loss
        loss_fn = nn.CrossEntropyLoss(reduction='mean', ignore_index=0)
        loss = loss_fn(logits.view(-1, config.vocab_size), targets.view(-1))
        running_loss += loss
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    print("====================Training===================")
    print('Patient:')
    print(tokenizer.decode(inputs['input_ids'].squeeze(1)[0],skip_special_tokens=True))
    print('Docter True:')
    print(tokenizer.decode(targets[0].squeeze(0),skip_special_tokens=True))
    print('Docter Pred:')
    print(tokenizer.decode(torch.argmax(logits, dim=-1)[0],skip_special_tokens=True))
    print("===============================================")

    total_loss_test = 0
    with torch.no_grad():
        for i,(inputs, targets) in enumerate(testloader):
            inputs,targets = inputs.to(device),targets.to(device)
            targets = targets.squeeze(1)
            inputs['input_ids'] = inputs['input_ids'].squeeze(1)
            inputs['token_type_ids'] = inputs['token_type_ids'].squeeze(1)
            inputs['attention_mask'] = inputs['attention_mask'].squeeze(1)

            outputs = model(**inputs)
            logits = linear_layer(outputs.last_hidden_state)

            ans_seq_len = targets.shape[1]
            logits = logits[:, :ans_seq_len, :] # Changed

            loss = loss_fn(logits.reshape(-1, logits.size(-1)), targets.reshape(-1))
            total_loss_test += loss.item()

            if i == 0 :
                print("====================Testing===================")
                print('Patient:')
                print(tokenizer.decode(inputs['input_ids'].squeeze(1)[0],skip_special_tokens=True))
                print('Docter True:')
                print(tokenizer.decode(targets[0],skip_special_tokens=True))
                print('Docter Pred:')
                print(tokenizer.decode(torch.argmax(logits, dim=-1)[0],skip_special_tokens=True))
                print("===============================================")

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {running_loss/len(dataloader)}")
    print(f"Epoch {epoch+1}/{num_epochs}, Test  Loss: {total_loss_test/len(testloader)}")
        # Add your code to track loss, accuracy, etc.


====================Training===================
Patient:
sos is it necessary to get tested for the corona virus if you arrive back in south africa after visiting zimbabwe and victoria falls as part of a group? eos sos is it necessary to get tested for the corona virus if you arrive back in south africa after visiting zimbabwe and victoria falls as part of a group? eos
Docter True:
sos in brief : it depends. if the visited place has a high rate of coronavirus infections to take some precautions is important such as if you develop fever or cough or any other coronavirus infection symptoms. to be tested is desired but will you be able to get tested? in some places testing kits are reserved to very sick and hospitalized patient. self quarantine in cooperation with your doctor is the way to go. would you like to video or text chat with me? eos
Docter Pred:
brief brief brief brief brief brief brief brief brief brief brief brief brief brief brief brief brief brief brief brief brief brief brie